In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/데이터전처리

/content/drive/MyDrive/데이터전처리


In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import random
import pprint # 보기 쉽게 깔끔한 형태로 출력

from transformers import AutoModel, AutoTokenizer

from torch import nn # 신경망(neural network)생성
import torch.nn.functional as F # 활성화함수, 손실함수를 대체하여 코드를 간결하게 만듬

from sklearn.metrics import accuracy_score, f1_score

In [ ]:
data = pd.read_excel("./data/data.xlsx", engine="openpyxl")#.iloc[:100]
# data["label_idx"] = [random.choice([0,1,2,3,4]) for _ in range(100)]
display(data.head(2))
data.shape

,sentence,label_idx
0,네,0
1,네 접시 색깔은 다른데 가격은 똑같아요,1997


(15726, 2)

In [ ]:
all_indices = data.index

In [ ]:
train_size = int(0.8 * len(all_indices)) # data의 80% 훈련용
val_size = len(all_indices) - train_size # data의 20% 검증용
train_idx, val_idx = all_indices[:int(0.8 * len(all_indices))], all_indices[int(0.8 * len(all_indices)):]

num_classes = max(data["label_idx"].tolist())+1 # data 개수 == 인덱스 최대값 + 1

In [ ]:
train_idx, val_idx

(RangeIndex(start=0, stop=12580, step=1),
 RangeIndex(start=12580, stop=15726, step=1))

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from transformers import get_linear_schedule_with_warmup # 학습 스케줄러를 생성
from tqdm.auto import tqdm # 프로그램 실행 진행사항 확인

# transformers 라이브러리에서 BERT 토크나이저를 임포트합니다.
from transformers import BertTokenizer

# 다국어 지원 BERT 모델을 사용하여 토크나이저 객체를 생성합니다.
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")

- BERT 입력형태
  - [CLS] 토큰 + 첫번째 문장 + [SEP] 토큰 + 두번째 문장
    - 문장의 시작과 끝, 문장 간의 구분을 명확하게 인식 가능
  - [CLS] 토큰
    - "Classification token", 입력 문장의 시작 부분에 추가.
    - 이 토큰의 임베딩은 문장 분류 작업에 사용, 문장 전체의 문맥을 요약하는 역할
  - [SEP] 토큰
    - "Separator token", 두 문장을 구분하는 데 사용
  - 임베딩
    - 텍스트 데이터를 컴퓨터가 이해하고 처리할 수 있는 수치형 벡터로 변환하는 과정 혹은 변환된 벡터


In [ ]:
# gpu사용할 수 있는지 확인(사용O: cuda, 사용X: cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터셋 클래스 정의
class TextClassificationDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.encodings = [
            # - tokenizer(train_data[’sentence’].tolist()[0]) : batch 형태로 입력을 줌
            # BERT에 필요한 입력 형태로 변환, 문장을 최대 길이에 맞게 패딩하고 결과값을 딕셔너리로 출력
            # padding: 문장 길이 동일하게 만들기 위해 패딩 추가, truncation: 최대길이 초과하는 경우 잘라냄
            tokenizer.encode_plus(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512) for sentence in df["sentence"].tolist()
        ]
        self.labels = df["label_idx"].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # to_return = {
        #     "sentence_encoded":self.encodings[idx],
        #     "label":self.labels[idx]
        # }

        # 인코딩된 문장, 레이블 쌍을 반환
        return (self.encodings[idx], self.labels[idx])
        # return to_return

In [ ]:
train_data = data.iloc[train_idx]
val_data = data.iloc[val_idx]

In [ ]:
train_dataset = TextClassificationDataset(train_data, tokenizer)
val_dataset = TextClassificationDataset(val_data, tokenizer)

In [ ]:
train_dataset[0]

({'input_ids': tensor([[  0, 752,   2]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])},
 0)

In [ ]:
batch_size = 1

# DataLoader : PyTorch에서 데이터 로드하는데 사용하는 클래스
# 데이터셋을 batch_size만큼 분할하여 로드
# shuffle=True: epoch마다 데이터셋을 무작위로 섞음(과적합 방지)
# shuffle=False: 데이터셋을 섞지않음(검증 데이터는 순서가 결과에 영향을 미치지 않기때문)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# 사전학습된 모델 불러옴
# klue/roberta-samll: KLUE(Korean Language Understanding Evaluation) 벤티마크를 위해 학습된 Roberta 모델의 작은 버전
model = AutoModel.from_pretrained("klue/roberta-small")

# 사용자 정의 모델 정의
class CustomModel(nn.Module):
    # hidden_size: hidden layer의 차원, 모델이 생성하는 벡터(임베딩)의 길이
    # 클수록 더 많은 정보를 담을 수 있지만, 과적합 위험이 높음 + 계산 복잡도와 메모리 사용량 증가
    def __init__(self, bert_model, num_classes, hidden_size):
        super(CustomModel, self).__init__()
        self.model = bert_model
        self.num_classes = num_classes

        # 선형 변환을 수행하는 레이어
        # BERT 모델의 출력을 입력받아 클래스의 수만큼 술력을 생성
        self.fc = nn.Linear(hidden_size, num_classes)

    # 모델 순전파
    # BERT 모델을 통해 입력을 전달, 출력을 선형 레이어를 통해 전달하여 최종 출력을 생성
    def forward(self, _input):
        # pooler_output: 문장의 전체적인 의미를 압축적으로 담음
        # [batch_size, hidden_size]형태의 텐서
        output = self.model(**_input)["pooler_output"]
        output = self.fc(output[0])
        return output


custom_model = CustomModel(model, num_classes, 768)
# AdamW: Adam optimizer의 변형, 가중치 감소(weight decay)를 지원
# custom_model의 파라미터를 최적화
optimizer = AdamW(custom_model.parameters(), lr=0.001)

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
loss_f = nn.CrossEntropyLoss()
epochs = 3

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    print('-' * 10)
    # 모델을 학습 모드로 설정합니다.
    model.train()
    # 입력 데이터와 레이블 전처리
    for i, (_input, _label) in enumerate(train_loader):
        # 각 토큰의 인코딩을 포함하는 딕셔너리
        _input = {k:v.squeeze(0) for k, v in _input.items()}
        # 각 레이블을 원핫 인코딩으로 변환하고 텐서로 변환
        _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()

        outputs = custom_model(_input)
        loss = loss_f(F.softmax(outputs), _label)
        loss.backward() # 역전파를 수행합니다.
        optimizer.step() # optimizer를 사용하여 파라미터 업데이터
        optimizer.zero_grad() # 그라디언트 초기화

    print(outputs)

    # 모델을 평가 모드로 설정합니다.
    model.eval()
    all_predictions = []
    all_labels = []

    for i, (_input, _label) in enumerate(val_loader):
        _input = {k:v.squeeze(0) for k, v in _input.items()}
        _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()

        outputs = custom_model(_input)
        #loss = loss_f(F.softmax(outputs), _label)

        # 모델의 예측과 실제 레이블을 저장
        # 예측은 소프트맥스 함수로 확률 변환하고 가장 높은 확률을 가진 클래스를 선택
        all_predictions.append( torch.argmax(F.softmax(outputs)).item() )
        all_labels.append( torch.argmax(F.softmax(_label)).item() )


    # 정확도와 가중치 평균 F! score 계산하여 출력
    print(accuracy_score(all_labels, all_predictions))
    print(f1_score(all_labels, all_predictions, average="weighted"))


<ipython-input-44-d9d816a47877>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()
<ipython-input-44-d9d816a47877>:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = loss_f(F.softmax(outputs), _label)


Epoch 1/3
----------
tensor([22.3561, -1.8212, -1.5098,  ..., -1.7176, -1.4976, -1.5368],
       grad_fn=<ViewBackward0>)


<ipython-input-44-d9d816a47877>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()
<ipython-input-44-d9d816a47877>:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  all_predictions.append( torch.argmax(F.softmax(outputs)).item() )
<ipython-input-44-d9d816a47877>:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  all_labels.append( torch.argmax(F.softmax(_label)).item() )


0.2463445645263827
0.09738180949142904
Epoch 2/3
----------


<ipython-input-44-d9d816a47877>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()
<ipython-input-44-d9d816a47877>:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = loss_f(F.softmax(outputs), _label)


/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3637145079.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()
/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3637145079.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  all_predictions.append( torch.argmax(F.softmax(outputs)).item() )
/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3637145079.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  all_labels.append( torch.argmax(F.softmax(_label)).item() )


In [ ]:
# 학습하기 이전 inference 결과
outputs

In [ ]:
# 학습하기 이전 inference 결과
outputs

In [ ]:
_label

In [ ]:
F.softmax(outputs)

/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3142704713.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(outputs)


tensor([8.3459e-08, 7.8275e-08, 1.0000e+00, 1.9766e-07, 1.6954e-07],
       grad_fn=<SoftmaxBackward0>)

In [ ]:
torch.argmax(F.softmax(outputs)).item()

/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_52307/3085399197.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.argmax(F.softmax(outputs)).item()


2

tensor([-3.6564, -3.7143, 11.9799, -2.7231, -2.9375], grad_fn=<ViewBackward0>)